Hello everyone!

I recently joined a competition and decided to explore the table data provided. I think it can also be helpful in solving and understanding the problem. Wanna share my notes.

It is my first experience applying theory in statistic. So correct me if i make something wrong.

Hope this kernel will be helpful for you!

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
class config:
    data_root = '../input/siim-isic-melanoma-classification'

### Read data

In [ ]:
train = pd.read_csv(config.data_root + '/train.csv')
test = pd.read_csv(config.data_root + '/test.csv')

In [ ]:
print('Size of train data: ', train.shape)
print('Size of test data: ', test.shape)

In [ ]:
train.head(5)

In [ ]:
test.head(5)

### Let's look at data

In [ ]:
print('Cases of melanoma in train data',len(train[train.target == 1]) / len(train) * 100,'%')

In [ ]:
print('Male in train data:',len(train[train.sex == 'male'])/len(train))
print('Male in test data:',len(test[test.sex == 'male'])/len(test))

In [ ]:
print('Unique patients in train data:', len(train.patient_id.unique())/len(train))
print('Unique patients in test data:', len(test.patient_id.unique())/len(test))

In [ ]:
print('Matual patients count',len(set(list(train.patient_id.unique())).intersection(list((test.patient_id.unique())))))

We do not have very many unique patients and all patients in the train and test are different.

We may also notice that skipped age and gender are found together:

In [ ]:
train[train.sex.isnull()].head(8)

In [ ]:
len(train[(train.sex.isnull()) & (train.age_approx.isnull())]) == len(train[train.sex.isnull()])

Only two people need this information

In [ ]:
train[(train.sex.isnull()) & (train.age_approx.isnull())].patient_id.unique()

Anatom_site_general_challenge feature

In [ ]:
len(train[train.anatom_site_general_challenge.isnull()])/len(train)

In [ ]:
train.anatom_site_general_challenge.unique()

So, we replace missing values with mean value and random value. 

For missing site we create new value  - unknown

In [ ]:
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].fillna('unknown')
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].fillna('unknown')

train['age_approx'] = train['age_approx'].fillna(train['age_approx'].mean())
test['age_approx'] = test['age_approx'].fillna(test['age_approx'].mean())

train['sex'] = train['sex'].fillna('female')
test['sex'] = test['sex'].fillna('male')

In [ ]:
train[train.isnull().any(1)]

In [ ]:
test[test.isnull().any(1)]

## Data Analysis

In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(1,2,1)
sns.distplot(train.age_approx,fit = norm)
plt.title('Age distribution in train data')
plt.subplot(1,2,2)
sns.distplot(test.age_approx,fit = norm)
plt.title('Age distribution in test data')
plt.show()

Overall, the age distribution in the data is close to normal distribution.

Сreate table with information about each patient:

In [ ]:
patients_age = pd.DataFrame(train.groupby(['patient_id'])['age_approx'].max())
patients_age.head(5)

In [ ]:
patients_disease = pd.DataFrame(train.groupby(['patient_id'])['target'].sum())
patients_disease.head(5)

In [ ]:
patients_gender = pd.DataFrame(train.groupby(['patient_id'])['sex'].max())
patients_gender.head(5)

In [ ]:
patients_history = patients_age.merge(patients_disease, on = 'patient_id')
patients_history = patients_history.merge(patients_gender, on = 'patient_id')
patients_history.head(5)

### How are age and disease risk related?

So, we have table where is information about patient. 
I want to test the effect of age on melanoma risk.

First check age distribution in groups with people without melanoma and with this disease.

In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(1,2,1)
sns.distplot(patients_history[patients_history.target == 0].age_approx,fit = norm)
plt.title('Benign cases')
plt.subplot(1,2,2)
sns.distplot(patients_history[patients_history.target != 0].age_approx,fit = norm)
plt.title('Melanoma')
plt.show()

### Q-Q Plot
https://en.wikipedia.org/wiki/Q–Q_plot

In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(1,2,1)
stats.probplot(patients_history[patients_history.target == 0].age_approx, plot=plt)
plt.subplot(1,2,2)
stats.probplot(patients_history[patients_history.target != 0].age_approx, plot=plt)
plt.show()

In [ ]:
print('M0 = ',patients_history[patients_history.target == 0].age_approx.mean())
print('M1 = ',patients_history[patients_history.target != 0].age_approx.mean())

print('D0 = ',patients_history[patients_history.target == 0].age_approx.var())
print('D1 = ',patients_history[patients_history.target != 0].age_approx.var())

### Student's t-test

https://en.wikipedia.org/wiki/Student%27s_t-test

Sample distributions are normal and we can use Student's t-test. We test hypothesis 0 that the means in these samples are equal.

H0: M0=M1

H1: M0!=M1



In [ ]:
from scipy.stats import ttest_ind

In [ ]:
t,p = ttest_ind(patients_history[patients_history.target == 0].age_approx,patients_history[patients_history.target != 0].age_approx)

print('p-value = ', p)
print('p-value < 0.05 - ', p<0.05)

p-value < 0.05 means that we reject the hypothesis of equality of means. Indeed, age is one of the susceptible risk factors, so we observe statistically significant differences.

### Test connection between the location of a malignant mole and age

I think that some areas of the body may be more at risk for sunburn. Thus, the age of patients with disease at different sites may differ.

In [ ]:
train[train.target == 1].groupby(['anatom_site_general_challenge'])['target'].count()

In [ ]:
plt.figure(figsize=(15, 15))

for i,site in enumerate(list(train.anatom_site_general_challenge.unique())):
    plt.subplot(3,3,i+1)
    sns.distplot(train[(train.anatom_site_general_challenge == site) & (train.target == 1)].age_approx,fit = norm)
    plt.title('Age distribution with {} melanoma'.format(site))
plt.show()

I will use only four sites: 'head/neck', 'upper extremity','lower extremity', 'torso'

In [ ]:
head_age = train[(train.target == 1) & (train.anatom_site_general_challenge == 'head/neck')].age_approx
upper_age = train[(train.target == 1) & (train.anatom_site_general_challenge == 'upper extremity')].age_approx
lower_age = train[(train.target == 1) & (train.anatom_site_general_challenge == 'lower extremity')].age_approx
torso_age = train[(train.target == 1) & (train.anatom_site_general_challenge == 'torso')].age_approx

## One-way ANOVA test

> The one-way ANOVA tests the null hypothesis that two or more groups have the same population mean. The test is applied to samples from two or more groups, possibly with different sizes.

https://en.wikipedia.org/wiki/One-way_analysis_of_variance

In [ ]:
(F,p_value) = stats.f_oneway(head_age,upper_age,lower_age,torso_age)
print('\nF = {:.2f}, p-value = {:.4f}'.format(F,p_value)) 

p - value < 0.05 indicates that there are statistically significant differences between at least two groups.

### Visualisation

In [ ]:
v = [head_age,upper_age,lower_age,torso_age]
avg = [a.mean() for a in v]
err = [1.96 * (np.std(a)/np.sqrt(len(a))) for a in v]

In [ ]:
plt.figsize = (30,45)
plt.errorbar(x = ['head/neck','upper extremity','lower extremity','torso'], y = avg, yerr = err, color="black", capsize=3, marker="s", markersize=5, mfc="red", mec="black",fmt = 'o')
plt.title('The age of the patient with melanoma at various sites')
plt.grid()
plt.xlabel('Site')
plt.ylabel('Age')
plt.show()

***Multiple comparisons problem*** occurs when we compare many pairs of samples

https://en.wikipedia.org/wiki/Multiple_comparisons_problem

### Bonferroni correction 

https://en.wikipedia.org/wiki/Bonferroni_correction

In [ ]:
p = 0.05
n = 4

c = (n-1)*n/2
p = p / c

print("new p = {:.10f}".format(p))

In [ ]:
site_target = train[train.target == 1][(train.anatom_site_general_challenge == 'head/neck') | (train.anatom_site_general_challenge == 'upper extremity') | (train.anatom_site_general_challenge == 'lower extremity') | (train.anatom_site_general_challenge == 'torso')]

###  TukeyHSD

https://en.wikipedia.org/wiki/Tukey%27s_range_test

In [ ]:
import statsmodels.stats.multicomp as multi

test = multi.MultiComparison(site_target['age_approx'], site_target['anatom_site_general_challenge'])
res = test.tukeyhsd(alpha = p)
summary = res.summary()
summary

We see that the age is different in the two pairs of samples. It can be assumed that the site plays an important role in the appearance of melanoma.